<a href="https://colab.research.google.com/github/kwagdm/project_compas/blob/master/%EC%8A%A4%ED%85%8C%EC%9D%B4%EC%85%98_%EC%A3%BC%EC%9C%84_%EC%9D%B8%EA%B5%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from shapely import geometry
from shapely.geometry import Point, Polygon
!pip install utm
import utm
!pip install geojson 

In [ ]:
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Point

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 10.9MB 372kB/s 
     |████████████████████████████████| 14.7MB 257kB/s 


In [ ]:
import pandas as pd
!pip install geopandas

In [ ]:
Bike_Station=pd.read_csv('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/02.자전거스테이션.csv')

In [ ]:
import geopandas as gpd
population=gpd.read_file('/content/drive/My Drive/프로젝트/compas/SBJ_2007_001/06.인구(거주)분포도(100M X 100M).geojson')

In [ ]:
population.head()

,gid,val,geometry
0,다사268650,NaN,"MULTIPOLYGON (((126.66985 37.68162, 126.66984 ..."
1,다사268651,NaN,"MULTIPOLYGON (((126.66984 37.68252, 126.66983 ..."
2,다사268652,NaN,"MULTIPOLYGON (((126.66983 37.68342, 126.66982 ..."
3,다사268653,NaN,"MULTIPOLYGON (((126.66982 37.68432, 126.66981 ..."
4,다사268654,NaN,"MULTIPOLYGON (((126.66981 37.68522, 126.66980 ..."


In [ ]:
Bike_Station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530


In [ ]:
Bike_Station['좌표'] = ''
for i in range(len(Bike_Station)):
  Bike_Station['좌표'][i] = (Bike_Station['경도'][i], Bike_Station['위도'][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#자전거 station이 속해있는 인구 grid(100*100)을 nearby_grid column에 두었음.

from shapely.geometry import Point, Polygon

Bike_Station['nearby_grid'] = ''
for i in range(len(population)):
  for j in range(len(Bike_Station)):
    if population['geometry'][i].contains(Point(Bike_Station['좌표'][j])):
      Bike_Station['nearby_grid'][j] = population['geometry'][i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#자전거 station이 속해있는 인구 grid(100*100)을 nearby_grid column에 두었음.
Bike_Station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도,좌표,nearby_grid
0,101,어울림마을 701동 앞,20,37.654775,126.834584,"(126.83458449999999, 37.65477464)",(POLYGON ((126.8345265803905 37.65472135795286...
1,103,대림e-편한세상106동,20,37.660442,126.840377,"(126.8403767, 37.66044160000001)",(POLYGON ((126.8401468033465 37.66016089405783...
2,104,탄현마을8단지,25,37.698523,126.766042,"(126.76604170000002, 37.69852315)",(POLYGON ((126.7649411575381 37.69847298187587...
3,105,KT 덕양지사 앞,20,37.655244,126.839261,"(126.83926070000001, 37.65524370000001)",(POLYGON ((126.8390610449085 37.65474685193998...
4,106,원당역 앞 공영주차장,30,37.653410,126.842530,"(126.8425305, 37.6534097)",(POLYGON ((126.8424777922569 37.65296329083705...


In [ ]:
population.head()

,gid,val,geometry
0,다사268650,NaN,"MULTIPOLYGON (((126.66985 37.68162, 126.66984 ..."
1,다사268651,NaN,"MULTIPOLYGON (((126.66984 37.68252, 126.66983 ..."
2,다사268652,NaN,"MULTIPOLYGON (((126.66983 37.68342, 126.66982 ..."
3,다사268653,NaN,"MULTIPOLYGON (((126.66982 37.68432, 126.66981 ..."
4,다사268654,NaN,"MULTIPOLYGON (((126.66981 37.68522, 126.66980 ..."


In [ ]:
Bike_Station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도,좌표,nearby_grid
0,101,어울림마을 701동 앞,20,37.654775,126.834584,"(126.83458449999999, 37.65477464)",(POLYGON ((126.8345265803905 37.65472135795286...
1,103,대림e-편한세상106동,20,37.660442,126.840377,"(126.8403767, 37.66044160000001)",(POLYGON ((126.8401468033465 37.66016089405783...
2,104,탄현마을8단지,25,37.698523,126.766042,"(126.76604170000002, 37.69852315)",(POLYGON ((126.7649411575381 37.69847298187587...
3,105,KT 덕양지사 앞,20,37.655244,126.839261,"(126.83926070000001, 37.65524370000001)",(POLYGON ((126.8390610449085 37.65474685193998...
4,106,원당역 앞 공영주차장,30,37.653410,126.842530,"(126.8425305, 37.6534097)",(POLYGON ((126.8424777922569 37.65296329083705...


In [ ]:
# https://www.python2.net/questions-277591.htm 참고
Bike_Station['dist'] = ''
for i in range(len(Bike_Station)):
  Bike_Station['dist'][i] = utm.from_latlon(float(str(Bike_Station['nearby_grid'][i].centroid)[7:-2].split()[1]), float(str(Bike_Station['nearby_grid'][i].centroid)[7:-2].split()[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
Bike_Station['700m_grid'] = ''

for i in range(len(Bike_Station)):
  p1 = utm.to_latlon(Bike_Station['dist'][i][0] - 350, Bike_Station['dist'][i][1] - 350, 52, 'S')
  p2 = utm.to_latlon(Bike_Station['dist'][i][0] + 350, Bike_Station['dist'][i][1] - 350, 52, 'S')
  p3 = utm.to_latlon(Bike_Station['dist'][i][0] - 350, Bike_Station['dist'][i][1] + 350, 52, 'S')
  p4 = utm.to_latlon(Bike_Station['dist'][i][0] + 350, Bike_Station['dist'][i][1] + 350, 52, 'S')

  pointList = [p1, p2, p4, p3]
  Bike_Station['700m_grid'][i] = geometry.Polygon(pointList)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
Bike_Station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도,좌표,nearby_grid,dist,700m_grid
0,101,어울림마을 701동 앞,20,37.654775,126.834584,"(126.83458449999999, 37.65477464)",(POLYGON ((126.8345265803905 37.65472135795286...,"(309028.44793811475, 4169761.812918777, 52, S)","POLYGON ((37.65194990351438 126.8312160004611,..."
1,103,대림e-편한세상106동,20,37.660442,126.840377,"(126.8403767, 37.66044160000001)",(POLYGON ((126.8401468033465 37.66016089405783...,"(309538.1912348699, 4170353.976239694, 52, S)","POLYGON ((37.65738958411129 126.8368357749251,..."
2,104,탄현마을8단지,25,37.698523,126.766042,"(126.76604170000002, 37.69852315)",(POLYGON ((126.7649411575381 37.69847298187587...,"(303005.1587147694, 4174760.9154854203, 52, S)","POLYGON ((37.69569961632893 126.7616314221886,..."
3,105,KT 덕양지사 앞,20,37.655244,126.839261,"(126.83926070000001, 37.65524370000001)",(POLYGON ((126.8390610449085 37.65474685193998...,"(309428.55925867415, 4169755.4104762557, 52, S)","POLYGON ((37.65197551888606 126.8357502882412,..."
4,106,원당역 앞 공영주차장,30,37.653410,126.842530,"(126.8425305, 37.6534097)",(POLYGON ((126.8424777922569 37.65296329083705...,"(309725.4412635125, 4169550.553321051, 52, S)","POLYGON ((37.65019205124443 126.8391669789059,..."


In [ ]:
def reverse_coordinate(x):
  t = str(x)[10:-2].split()
  for i in range(len(t)):
    t[i] = t[i].strip(',')
  t[0], t[1] = t[1], t[0]
  t[2], t[3] = t[3], t[2]
  t[4], t[5] = t[5], t[4]
  t[6], t[7] = t[7], t[6]
  t[8], t[9] = t[9], t[8]
  t = (t[0] + ' ' + t[1]), (t[2] + ' ' + t[3]), (t[4] + ' ' + t[5]), (t[6] + ' ' + t[7]), (t[8] + ' ' + t[9])
  t2 = []
  for i in range(len(t)):
    t2.append(t[i].split())
  for i in range(len(t2)):
    for j in range(2):
      t2[i][j] = float(t2[i][j])
  t2
  return Polygon([(t2[0][0], t2[0][1]), (t2[1][0], t2[1][1]), (t2[2][0], t2[2][1]), (t2[3][0], t2[3][1]), (t2[4][0], t2[4][1])])




In [ ]:
Bike_Station['700m_grid_reverse'] = ''
for i in range(len(Bike_Station)):
  Bike_Station['700m_grid_reverse'][i] = reverse_coordinate(Bike_Station['700m_grid'][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
Bike_Station.head()

,Station_ID,STATION_NAME,거치대 수량,위도,경도,좌표,nearby_grid,dist,700m_grid,700m_grid_reverse
0,101,어울림마을 701동 앞,20,37.654775,126.834584,"(126.83458449999999, 37.65477464)",(POLYGON ((126.8345265803905 37.65472135795286...,"(309028.44793811475, 4169761.812918777, 52, S)","POLYGON ((37.65194990351438 126.8312160004611,...","POLYGON ((126.8312160004611 37.65194990351438,..."
1,103,대림e-편한세상106동,20,37.660442,126.840377,"(126.8403767, 37.66044160000001)",(POLYGON ((126.8401468033465 37.66016089405783...,"(309538.1912348699, 4170353.976239694, 52, S)","POLYGON ((37.65738958411129 126.8368357749251,...","POLYGON ((126.8368357749251 37.65738958411129,..."
2,104,탄현마을8단지,25,37.698523,126.766042,"(126.76604170000002, 37.69852315)",(POLYGON ((126.7649411575381 37.69847298187587...,"(303005.1587147694, 4174760.9154854203, 52, S)","POLYGON ((37.69569961632893 126.7616314221886,...","POLYGON ((126.7616314221886 37.69569961632893,..."
3,105,KT 덕양지사 앞,20,37.655244,126.839261,"(126.83926070000001, 37.65524370000001)",(POLYGON ((126.8390610449085 37.65474685193998...,"(309428.55925867415, 4169755.4104762557, 52, S)","POLYGON ((37.65197551888606 126.8357502882412,...","POLYGON ((126.8357502882412 37.65197551888606,..."
4,106,원당역 앞 공영주차장,30,37.653410,126.842530,"(126.8425305, 37.6534097)",(POLYGON ((126.8424777922569 37.65296329083705...,"(309725.4412635125, 4169550.553321051, 52, S)","POLYGON ((37.65019205124443 126.8391669789059,...","POLYGON ((126.8391669789059 37.65019205124443,..."


In [ ]:
population.head()

,gid,val,geometry
0,다사268650,NaN,"MULTIPOLYGON (((126.66985 37.68162, 126.66984 ..."
1,다사268651,NaN,"MULTIPOLYGON (((126.66984 37.68252, 126.66983 ..."
2,다사268652,NaN,"MULTIPOLYGON (((126.66983 37.68342, 126.66982 ..."
3,다사268653,NaN,"MULTIPOLYGON (((126.66982 37.68432, 126.66981 ..."
4,다사268654,NaN,"MULTIPOLYGON (((126.66981 37.68522, 126.66980 ..."


In [ ]:
population_station = []

for i in range(len(Bike_Station)):
  for j in range(len(population)):   
    pop_count = 0 
    if Bike_Station['700m_grid_reverse'][i].contains(population['geometry'][j]):
      if population['val'][j] == 'nan':
        continue
      else:
        pop_count += population['val'][j]

      population_station.append(pop_count)

In [ ]:
population_station

[15.0,
 187.0,
 139.0,
 103.0,
 0.0,
 nan,
 50.0,
 34.0,
 202.0,
 719.0,
 nan,
 8.0,
 291.0,
 21.0,
 316.0,
 624.0,
 17.0,
 365.0,
 46.0,
 885.0,
 501.0,
 144.0,
 89.0,
 404.0,
 225.0,
 365.0,
 0.0,
 271.0,
 37.0,
 261.0,
 357.0,
 15.0,
 142.0,
 301.0,
 300.0,
 168.0,
 360.0,
 547.0,
 816.0,
 nan,
 137.0,
 537.0,
 173.0,
 268.0,
 458.0,
 723.0,
 nan,
 405.0,
 376.0,
 273.0,
 425.0,
 860.0,
 229.0,
 953.0,
 517.0,
 363.0,
 nan,
 6.0,
 542.0,
 583.0,
 467.0,
 nan,
 nan,
 nan,
 906.0,
 464.0,
 983.0,
 855.0,
 nan,
 0.0,
 nan,
 nan,
 681.0,
 297.0,
 nan,
 nan,
 nan,
 741.0,
 1083.0,
 839.0,
 0.0,
 0.0,
 0.0,
 285.0,
 675.0,
 411.0,
 781.0,
 727.0,
 280.0,
 479.0,
 0.0,
 796.0,
 nan,
 459.0,
 757.0,
 nan,
 nan,
 773.0,
 183.0,
 89.0,
 404.0,
 880.0,
 365.0,
 0.0,
 271.0,
 37.0,
 261.0,
 435.0,
 15.0,
 142.0,
 165.0,
 0.0,
 28.0,
 558.0,
 519.0,
 43.0,
 6.0,
 52.0,
 nan,
 18.0,
 19.0,
 12.0,
 321.0,
 407.0,
 nan,
 478.0,
 453.0,
 388.0,
 376.0,
 261.0,
 nan,
 219.0,
 519.0,
 43.0,
 nan,
 nan

In [ ]:

for j in range(len(population)):
  pop_count = 0
  if Bike_Station['700m_grid_reverse'][0].contains(population['geometry'][j]):
      if population['val'][j] == 'nan':
        continue
      else:
        pop_count += population['val'][j]

In [ ]:
pop_count

0